<a href="https://colab.research.google.com/github/Samruddhi-saoji/Deep-Learning-self-study-codes/blob/main/pre_trained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, classification_report

### Dataset Handler

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory


class DataLoader:
    def __init__(self, train_path="", validation_path="", test_path="") -> None:
        # set the parameters
        self.train_path = train_path
        self.test_path = test_path
        self.vali=False #does vali set exist?
        self.vali_path = ""
        if validation_path != "":
            self.vali = True #bool
            self.vali_path = validation_path

        # these values will be calculated when data is loaded
        self.classes = []
        self.num_classes = -1


    # label_type = "categorical" for onehot encoding
    def load_data(self, image_shape, label_type='int', batch_size=32) -> None:
        #load the datasets
        self.train_dataset = image_dataset_from_directory(
            self.train_path,
            image_size=image_shape,
            batch_size=batch_size,
            label_mode=label_type
        )

        self.test_dataset = image_dataset_from_directory(
            self.test_path,
            image_size=image_shape,
            batch_size=batch_size,
            label_mode=label_type
        )

        if self.vali_path != "":
                self.vali_dataset = image_dataset_from_directory(
                self.vali_path,
                image_size=image_shape,
                batch_size=batch_size,
                label_mode=label_type
            )

        # get the classes
        self.classes = self.train_dataset.class_names
        self.num_classes = len(self.classes)

        # get X, Y
        X_train, Y_train = self.get_X_and_Y(self.train_dataset)
        X_test, Y_test = self.get_X_and_Y(self.test_dataset)
        X_vali, Y_vali = [], []
        if self.vali:
            X_vali, Y_vali = self.get_X_and_Y(self.vali_dataset)

        return X_train, Y_train, X_vali, Y_vali, X_test, Y_test



    # Convert datasets into arrays (optional, for X_train, Y_train, etc.)
    def get_X_and_Y(self, dataset):
        images = []
        labels = []
        for image_batch, label_batch in dataset:
            images.append(image_batch.numpy())
            labels.append(label_batch.numpy())
        return np.concatenate(images), np.concatenate(labels)

# Data

fruits 360 dataset

Considering only these 7 classes:
 - Golden apple 1
 - Avocado
 - Lemon
 - Mango
 - Kiwi
 - Banana
 - Strawberry
 - Raspberry

## Load the dataset

In [ ]:
## mount google drive

from google.colab import drive
drive.mount('/content/drive')

# Done once, no need to do it again

Mounted at /content/drive


In [ ]:
img_shape = (100, 100)
train_path = "/content/drive/MyDrive/Datasets/kaggle fruits 360/fruits limited dataset/Training"
test_path = "/content/drive/MyDrive/Datasets/kaggle fruits 360/fruits limited dataset/Test"
dataset = DataLoader(train_path=train_path, test_path=test_path)
X_train, Y_train, _, __, X_test, Y_test = dataset.load_data(image_shape=img_shape)

# shuffle data
X_train, Y_train = shuffle(X_train, Y_train, random_state=42)
X_test, Y_test = shuffle(X_test, Y_test, random_state=42)

# dont do feature scaling for vgg
'''#feature scaling
X_train = X_train/255.0
X_test = X_test/255.0
'''

Found 3827 files belonging to 8 classes.
Found 1285 files belonging to 8 classes.


'#feature scaling\nX_train = X_train/255.0\nX_test = X_test/255.0\n'

## Preprocess data

- Keras provides an inbuilt function to handle the data preprocessing.
- Each pretrained model has diff recquirements, thus each has a diff preprocessing function

In [ ]:
# select correct model name
# from keras.applications.vgg16 import preprocess_input
from keras.applications.resnet import preprocess_input

# Preprocess data
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

# Pre-trained models

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Flatten #layers

In [ ]:
# Plot loss over the epochs
def plot_loss(info):
    plt.plot(info.history['loss'], label='Training loss', color='r')
    plt.plot(info.history['val_loss'], label='Validation loss', color='b')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

## Vgg

In [ ]:
from keras.applications.vgg16 import preprocess_input, VGG16 #pre-trained model

In [ ]:
''' Build the model '''

input_shape = (100, 100, 3)

# the base (convo-pooling) layers
base = VGG16(input_shape=input_shape, weights="imagenet", include_top=False)

# freeze all the base layers
for layer in base.layers:
   layer.trainable = False

# add Flattening and Dense layers to the base model
x = Flatten()(base.output)
x = Dense(10, activation='relu')(x)
y = Dense(8, activation='softmax')(x) #predicted output

# create the final model
model = Model(inputs=base.input, outputs=y)

# compile the model
model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
            )

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Resnet

In [ ]:
from keras.applications.resnet50 import ResNet50

In [ ]:
''' Build the model '''

input_shape = (100, 100, 3)

# the base (convo-pooling) layers
base = ResNet50(input_shape=input_shape, weights="imagenet", include_top=False)

# freeze all the base layers
for layer in base.layers:
   layer.trainable = False

# add Flattening and Dense layers to the base model
x = Flatten()(base.output)
x = Dense(10, activation='relu')(x)
y = Dense(dataset.num_classes, activation='softmax')(x) #predicted output

# create the final model
model = Model(inputs=base.input, outputs=y)

# compile the model
model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
            )

## Inception

## Alexnet

## YOLO (You Only Look Once)

# Training and testing

In [ ]:
''' Training and testing'''

# Training and validation
e = 3 #epochs
info = model.fit(X_train, Y_train, epochs=e, validation_split=0.15)

# plot train and vali loss
plot_loss(info)

# Testing
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f"Test accuracy:{test_accuracy:.4f}")